## First Attempt at building a deepsets architecture

In [1]:
import random

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from astropy.io import fits
import healpy as hp



#### Experimenting with input sizes

In [ ]:
# For DECAM, BASS, MzLS
with open('../../bricks_data/pixel2ccd_2048_non_inclusive.pickle', 'rb') as f:
    subpixel2ccd_dict = pickle.load(f)
    f.close()

In [ ]:
print(len(subpixel2ccd_dict))

In [ ]:
min = 1000
max = 0
aggregate = 0
index_max_sub = 0
lens = np.zeros(len(subpixel2ccd_dict))

for i,pix in enumerate(subpixel2ccd_dict.keys()):
    no_ccds = len(subpixel2ccd_dict[pix])
    lens[i] = no_ccds
    aggregate += no_ccds
    if no_ccds > max:
        max = no_ccds
        index_max_sub = pix
    if no_ccds < min:
        min = no_ccds

print("Mean CCD's per subpixel:", lens.mean())
print("Max CCD's per subpixel:", max)
print("Min CCD's per subpixel:", min)



In [ ]:
plt.hist(lens, bins=[0,30,50,100])

In [ ]:
with open('../../bricks_data/pixel2ccd_256_non_inclusive.pickle', 'rb') as f:
    pixel2ccd_dict = pickle.load(f)
    f.close()

In [ ]:
min = 1000
max = 0
aggregate = 0
index_max = 0
lens = np.zeros(len(pixel2ccd_dict))

for i,pix in enumerate(pixel2ccd_dict.keys()):
    no_ccds = len(pixel2ccd_dict[pix])
    lens[i] = no_ccds
    aggregate += no_ccds
    if no_ccds > max:
        max = no_ccds
        index_max = pix
    if no_ccds < min:
        min = no_ccds

print("Mean CCD's per pixel:", lens.mean())
print("Max CCD's per pixel:", max)
print("Min CCD's per pixel:", min)


In [ ]:
plt.hist(lens, bins=[0,30,50,100])

In [ ]:
decamCCD = fits.open('../../bricks_data/ccds-annotated-decam-dr9.fits')
mosaicCCD = fits.open('../../bricks_data/ccds-annotated-mosaic-dr9.fits')
bassCCD = fits.open('../../bricks_data/ccds-annotated-90prime-dr9.fits')
#print(decamCCD[1].columns)


In [ ]:
dataDecam = decamCCD[1].data
dataMosaic = mosaicCCD[1].data
dataBass = bassCCD[1].data


In [ ]:
ra0 = np.concatenate((dataDecam.field('ra0'), dataMosaic.field('ra0'), dataBass.field('ra0')), axis=0)
dec0 = np.concatenate((dataDecam.field('dec0'), dataMosaic.field('dec0'), dataBass.field('dec0')), axis=0)

ra1 = np.concatenate((dataDecam.field('ra1'), dataMosaic.field('ra1'), dataBass.field('ra1')), axis=0)
dec1 = np.concatenate((dataDecam.field('dec1'), dataMosaic.field('dec1'), dataBass.field('dec1')), axis=0)

ra2 = np.concatenate((dataDecam.field('ra2'), dataMosaic.field('ra2'), dataBass.field('ra2')), axis=0)
dec2 = np.concatenate((dataDecam.field('dec2'), dataMosaic.field('dec2'), dataBass.field('dec2')), axis=0)

ra3 = np.concatenate((dataDecam.field('ra3'), dataMosaic.field('ra3'), dataBass.field('ra3')), axis=0)
dec3 = np.concatenate((dataDecam.field('dec3'), dataMosaic.field('dec3'), dataBass.field('dec3')), axis=0)



### Plotting the Pixel - CCD Relationship for the most populated pixel

In [ ]:
#Validating everything went as intended visually

NSIDE = 256
pixel_number = index_max
# print(pixel2ccd_dict.keys())
print(len(pixel2ccd_dict.keys()))


ccd = pixel2ccd_dict[pixel_number]
print("Number of CCDs that are cutting pixel number ", pixel_number, ":", len(ccd))
for cc in ccd:
#coord = [[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]
    xs = [ra0[cc],ra1[cc], ra2[cc], ra3[cc],ra0[cc] ]
    ys = [dec0[cc],dec1[cc], dec2[cc], dec3[cc],dec0[cc]]
    plt.plot(xs,ys)
#if i > 3:
    #break

pixel_boundary = hp.boundaries(nside=NSIDE, pix=pixel_number, step=1)
pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(),lonlat=True)
pixel_boundary_raDec = list(pixel_boundary_raDec)
pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0],pixel_boundary_raDec[0][0])
pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1],pixel_boundary_raDec[1][0])


plt.plot(pixel_boundary_raDec[0],pixel_boundary_raDec[1], c='black', label="Pixel Boundary")

plt.show()

In [ ]:
#Validating everything went as intended visually

NSIDE = 2048
pixel_number = index_max_sub
# print(pixel2ccd_dict.keys())
print(len(subpixel2ccd_dict.keys()))


ccd_sub = subpixel2ccd_dict[pixel_number]
print("Number of CCDs that are cutting pixel number ", pixel_number, ":", len(ccd))
for cc in ccd_sub:
#coord = [[x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0]]
    xs = [ra0[cc],ra1[cc], ra2[cc], ra3[cc],ra0[cc] ]
    ys = [dec0[cc],dec1[cc], dec2[cc], dec3[cc],dec0[cc]]
    plt.plot(xs,ys)
#if i > 3:
    #break

pixel_boundary = hp.boundaries(nside=NSIDE, pix=pixel_number, step=1)
pixel_boundary_raDec = hp.vec2ang(pixel_boundary.transpose(),lonlat=True)
pixel_boundary_raDec = list(pixel_boundary_raDec)
pixel_boundary_raDec[0] = np.append(pixel_boundary_raDec[0],pixel_boundary_raDec[0][0])
pixel_boundary_raDec[1] = np.append(pixel_boundary_raDec[1],pixel_boundary_raDec[1][0])


plt.plot(pixel_boundary_raDec[0],pixel_boundary_raDec[1], c='black', label="Pixel Boundary")

plt.show()

In [ ]:
x = [x for x in ccd_sub if x not in ccd]

print(len(x))

## Building a deepsets architecture

In [2]:
# Import NN Packages
import torch
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn import preprocessing, metrics
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt


print(torch.__version__)


1.9.0


## Writing Dataset Utility to pass data in the right format

#### Steps:

1. Build a NN sampling an equal number of CCDs per 256 pixel and pass through deep sets for regression

2. Adapt NN for variable sized inputs

3. Adapt NN to use 64 inputs of size 2048 to then predict density at 256

In [ ]:
# Preparing Data to use
df_raw = pd.read_csv('../../bricks_data/dataset_geometric.csv')


In [ ]:
df_raw.head()

In [ ]:
num_pixels = 10
gal_type = 'lrg'
num_features = 5
max_ccds = 30

ids = random.sample(range(len(df_raw)),num_pixels)
print(ids)
df = df_raw.iloc[ids]
print(df.head())
labels = df[gal_type].to_numpy()
pix_ids = df.pixel_id.to_numpy()
print(labels)
print(pix_ids)




In [ ]:
ids = [4257974, 4842643, 4842704, 4949516, 5640882, 5668149, 5817353, 5823130, 5958363]

print(len(pixel2ccd_dict))
for i in pix_ids:
    print(len(pixel2ccd_dict[i]))
    print(pixel2ccd_dict[i])

from set_dataloader import CCD
ccd = CCD()
x = ccd.get_ccds(ids)

print(x)


In [ ]:
dataset = -1*np.ones((num_pixels,max_ccds,num_features))

# Iterate through the pixels
for i,pix in enumerate(pix_ids):
    ids = pixel2ccd_dict[pix]
    print(len(ids))
    ids = ids[:30]
    print(len(ids))
    print()
    x = ccd.get_ccds(ids)
    # Iterate through the CCDs for every pixel
    for j in range(len(ids)):
        dataset[i,j] = x[j]
print(dataset)
print()
print(dataset[3])
print()
print(dataset[3,2,2])

In [5]:
from set_dataloader import CCD
# noinspection PyAttributeOutsideInit
class SetSequence(Dataset):
    """Processes and Returns a Dataset of Variable Sized Input Sets of Dimensions
    N = Number Pixels of that are returned
    M = Max Size of each Individual Set of CCDs


    """

    def __init__(self, gal_type = 'lrg', num_pixels = 10, max_ccds = 30, var_set_len = False):

        with open('../../bricks_data/pixel2ccd_256_non_inclusive.pickle', 'rb') as f:
            self.pixel2ccd_dict = pickle.load(f)
            f.close()


        self.ccd = CCD()
        self.num_features = self.ccd.num_features

        self.gal_type = gal_type

        # Dimensions
        self.num_pixels = num_pixels
        self.max_ccds = max_ccds
        self.var_set_len = var_set_len

        df_raw = pd.read_csv('../../bricks_data/dataset_geometric.csv')
        # Randomly Sampling Pixel Indices from Dataframe
        pixel_indices = random.sample(range(len(df_raw)), num_pixels)
        print(pixel_indices)



        self.df = df_raw.iloc[pixel_indices]
        self.pix_ids = self.df.pixel_id.to_numpy()

        # Features and inputs:
        self.target = self.df[gal_type].to_numpy()


        self.initialise_inputs()

        self.initialise_lengths()

        print(self.lengths)
        # Target
        self.label = np.random.rand(self.num_pixels*self.max_ccds)


        # Mask Variable Len Sets
        #self.set_max_set_len()

    def initialise_lengths(self):
        self.lengths = np.zeros(self.num_pixels, dtype=int)
        if self.var_set_len:
            for i, pix in enumerate(self.pix_ids):
                c = len(self.pixel2ccd_dict[pix])
                if c < self.max_ccds:
                    self.lengths[i] = c
                else:
                    self.lengths[i] = self.max_ccds

        else:
            self.lengths.fill(self.max_ccds)

    def initialise_inputs(self):
        self.input = -1 * np.ones((self.num_pixels, self.max_ccds, self.num_features))
        # Iterate through the pixels
        for i, pix in enumerate(self.pix_ids):
            ids = self.pixel2ccd_dict[pix]
            print(len(ids))
            ids = ids[:30]
            print(len(ids))
            print()
            x = self.ccd.get_ccds(ids)
            # Iterate through the CCDs for every pixel
            for j in range(len(ids)):
                self.input[i, j] = x[j]

    def set_max_set_len(self):
        self.index_matrix =  -1*np.ones((self.num_pixels, self.max_ccds), dtype=int)

        # Getting random labels for now, in the future this will be the output densities

        m = 0
        for i in range(self.num_pixels):

            for j in range(self.lengths[i]):
                ''' This code with label == 0 is not yet needed, but this masking will become necessary when I have
                    I have 64 subpixels per pixel and some of those are not covered by CCDs'''
                while self.label[m] == 0:
                    m += 1
                self.index_matrix[i, j] = m
                m += 1

        print(self.lengths)
        print(self.index_matrix)

    def __len__(self):
        return self.num_pixels

    def __getitem__(self, idx):
        x = torch.from_numpy(self.input[idx]).float()
        y = torch.tensor(self.target[idx]).float()
        l = self.lengths[idx]
        return x,y,l


""" Todo
1. Where to get the data from
2. Scaling --> import an already scaled dataset, this will have to be prepared but should be same for Neural Net
3. Combine larger and smaller dataset
4. Build 64 input channels instead of one, so one more dimension of tensors( NO of Pixels,no_of_subpixels,no_ccds, no_features)
"""

In [6]:
traindata = SetSequence(var_set_len=True)
print("here")
x,y,l = traindata.__getitem__(4)
print(x)
print(y)
print(l)

[36438, 139395, 248569, 212638, 61172]
10
10

11
11

21
21

30
30

18
18

[10 11 21 30 18]
here
tensor([[  2.0000,   1.0000, 100.0000,   2.1900,   1.1626],
        [  2.0000,   1.0000, 250.0000,   2.2200,   1.0982],
        [  0.0000,   1.0000, 200.0000,   2.1800,   1.6697],
        [  1.0000,   1.0000,  67.0000,   2.1700,   1.1360],
        [  1.0000,   1.0000,  94.0000,   2.3800,   1.5646],
        [  0.0000,   1.0000, 200.0000,   2.1900,   1.6412],
        [  2.0000,   2.0000,  80.1030,   1.0031,   0.7991],
        [  2.0000,   2.0000,  80.1050,   1.0075,   0.8777],
        [  2.0000,   2.0000,  80.1040,   1.0046,   1.0297],
        [  2.0000,   2.0000, 113.1040,   1.0007,   1.6532],
        [  1.0000,   0.0000, 237.6000,   1.0200,   0.8096],
        [  0.0000,   0.0000, 145.0000,   1.0000,   0.7624],
        [  1.0000,   0.0000, 152.8000,   1.0200,   0.8725],
        [  0.0000,   0.0000,  93.9310,   1.0000,   0.7310],
        [  1.0000,   0.0000,  68.2910,   1.0100,   0.5528],
    

In [ ]:
class DensitySurvey(Dataset):
    def __init__(self, df, galaxy_type, scaler_in=None, scaler_out=None):
        self.data = df
        # Extracting Targets and Input
        self.target = self.data[galaxy_type].to_numpy(copy=True)
        self.input = self.data.drop(columns=['lrg','elg','qso']).to_numpy(copy=True)

        # Scaling, when scaler is passed (test-set) use the existing scaler
        self.scaler_in = scaler_in
        self.scaler_out = scaler_out
        if self.scaler_in is None:
            self.scaler_in = preprocessing.MinMaxScaler()
            self.scaler_out = preprocessing.MinMaxScaler()
            self.input = self.scaler_in.fit_transform(self.input)
            self.target = self.scaler_out.fit_transform(self.target.reshape(-1, 1))
        else:
            self.input = self.scaler_in.transform(self.input)
            self.target = self.scaler_out.transform(self.target.reshape(-1, 1))


    def __len__(self):
        return len(self.target)

    def __getitem__(self, idx):
        return torch.from_numpy(self.input[idx]).float(), torch.tensor(self.target[idx]).float()


In [ ]:
dens = DensitySurvey